In [2]:
%load_ext nvcc4jupyter

from nvcc4jupyter import set_defaults
set_defaults(compiler_args='-arch=sm_120 -Xptxas=-v -O0')

Source files will be saved in "/tmp/tmp8yvqaflh".


In [ ]:
%%cuda
#include<stdio.h> 
#include<cuda.h> 
#include<cuda_runtime.h> 
#include<cuda_fp16.h> 
#include<cuda_bf16.h>
#include <iostream>
#include <vector>
#include <random>

#define CHECK_CUDA(call) { \
    cudaError_t err = call; \
    if (err != cudaSuccess) { \
        fprintf(stderr, "CUDA Error: %s at line %d\n", cudaGetErrorString(err), __LINE__); \
        exit(1); \
    } \
}
enum InitMode
{
  MODE_ZEROS, 
  MODE_RAND, 
  MODE_ARANGE,
};

template <typename T> 
void init_array_host_to_device(T*& host_null_ptr, T*& device_null_ptr, int N_elems, InitMode mode)
{
  CHECK_CUDA(cudaHostAlloc(&host_null_ptr, N_elems*sizeof(T), cudaHostAllocDefault));
  switch (mode)
  {
    case MODE_ZEROS:
      break;
    case MODE_ARANGE: 
      for (int i = 0; i < N_elems; i++) host_null_ptr[i] = static_cast<T>(i);
      break;
    case MODE_RAND: 
    {
      std::random_device rd;
      std::mt19937 gen(rd());
      if constexpr (std::is_floating_point<T>::value || sizeof(T) == 2) {
         std::uniform_real_distribution<float> dist(-1.0f, 1.0f);
         for(int i = 0; i < N_elems; ++i) 
             host_null_ptr[i] = static_cast<T>(dist(gen));
      } else {
         std::uniform_int_distribution<int> dist(0, 100);
         for(int i = 0; i < N_elems; ++i) 
             host_null_ptr[i] = static_cast<T>(dist(gen));
      }
      break;
    }

  }

}

int main()
{
  int* A_h, *A_d; 
  A_h = nullptr; 
  A_d = nullptr;
  init_array_host_to_device(A_h, A_d, 10, MODE_RAND);
  printf("\n %f \n",(float)A_h[2]);
  
  return 0;
}


 15.000000 

